In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import time

In [ ]:
#FVariables auxiliares
SBU={ 'ANIO':[2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018],
     'sbu':[105, 122, 136, 150, 160, 170, 200, 218, 240, 264, 292, 318, 340, 354, 366, 375, 386]}

In [2]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
elif socket.gethostname()=='LAPTOP-PUSGG08B11': #Ip del working
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656.dsv'
# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656.dsv'

In [3]:
vej = pd.read_csv( ruta_vj, delimiter='\t', encoding='utf-8', decimal='.', dtype={'CEDULA_COD':'object'})
vej['FECHA_NACIMIENTO'] = pd.to_datetime(vej['FECHA_NACIMIENTO'], format='%d/%m/%Y')
vej['FECHA_MUERTE'] = pd.to_datetime(vej['FECHA_MUERTE'], format='%d/%m/%Y')
vej['FECHA_DERECHO'] = pd.to_datetime(vej['FECHA_DERECHO'], format='%d/%m/%Y')
vej['RANGO_INI_5MEJ'] = pd.to_datetime(vej['RANGO_INI_5MEJ'], format='%d/%m/%Y')
vej['RANGO_FIN_5MEJ'] = pd.to_datetime(vej['RANGO_FIN_5MEJ'], format='%d/%m/%Y')

In [4]:
#Datos de vejez
vej.sort_values(by='CEDULA_COD',inplace=True)

In [44]:
vej[vej['CEDULA_COD']=='100018']

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUM_IMPOSICIONES,COEFICIENTE_TEO,COEFICIENTE_REAL,BASE_AP_TEO,BASE_AP_REAL,PENSION,TIPO_PRESTACION,RANGO_INI_5MEJ,RANGO_FIN_5MEJ
471930,100018,H,1949-12-08,2020-10-06,2010-03-01,399,0.7875,0.7875,319.68,319.68,498.51,VEJEZ,2019-10-31,2002-03-01


In [6]:
#Se importa el archivo
afi = pd.read_csv(ruta_afi, delimiter='\t', encoding='utf-8', decimal='.', dtype={'CEDULA_COD':'object'})
#afi=afi.set_index(['CEDULA_COD'])
# afi[afi['CEDULA_COD']==114]

In [7]:
afi

,CEDULA_COD,ANIO,MES,SALARIO,PORCENTAJE_AP,SECTOR
0,20057738,2003,3,135.03,20.5,PRIVADO
1,15613923,2003,11,65.61,20.5,PRIVADO
2,16057398,1996,11,56.65,20.5,PRIVADO
3,16023482,1997,9,15.11,20.5,PRIVADO
4,16021285,1998,1,27.27,20.5,PUBLICO
...,...,...,...,...,...,...
66555199,16773012,2005,9,230.00,20.5,PRIVADO
66555200,5709849,2007,1,278.95,30.5,PUBLICO
66555201,19960695,2008,9,1400.00,20.5,PRIVADO
66555202,11118795,1993,3,56.65,20.5,PRIVADO


In [8]:
afi['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO', 'INDEPENDIENTES',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR', 'PASANTE PRIVADO',
       'PASANTE PUBLICO', '108-ARTISTA Y GESTOR DE CULTURA',
       '06-CODIGO DEL TRABAJO - CT '], dtype=object)

In [16]:
#Cedulas y aparición en cada sector
ced_sec = afi.groupby('CEDULA_COD')['SECTOR'].nunique()

In [199]:
ced_sec[ced_sec==2]

CEDULA_COD
1000543    2
1000545    2
1000661    2
100072     2
1000784    2
          ..
997632     2
99774      2
9980       2
99860      2
99916      2
Name: SECTOR, Length: 136187, dtype: int64

In [201]:
afi[afi['CEDULA_COD']=='100018']['SECTOR'].unique()

array(['PRIVADO'], dtype=object)

In [22]:
#Cédulas que tiene un único sector
uni_ced_sec = ced_sec[ced_sec == 1].index.tolist()
#Cédulas que tiene un dos sector
dos_ced_sec = ced_sec[ced_sec == 2].index.tolist()
#Cédulas que tiene un tres sector
tres_ced_sec = ced_sec[ced_sec == 3].index.tolist()
#Cédulas que tiene más tres sector
mas_ced_sec = ced_sec[ced_sec > 3].index.tolist()

In [23]:
#Afliados de HL con un único sector
afi_uni = afi[afi['CEDULA_COD'].isin(uni_ced_sec)]
#Afliados de HL con un único sector
afi_dos = afi[afi['CEDULA_COD'].isin(dos_ced_sec)]
#Afliados de HL con un único sector
afi_tres = afi[afi['CEDULA_COD'].isin(tres_ced_sec)]
#Afliados de HL con un único sector
afi_mas = afi[afi['CEDULA_COD'].isin(mas_ced_sec)]

In [24]:
#Se presenta la clasificación
print('Se tiene un total ',afi['CEDULA_COD'].nunique(), ' personas que se va analizar su HL')
print('Se tiene un total ',afi_uni['CEDULA_COD'].nunique(), ' personas que tienen un único sector')
print('Se tiene un total ',afi_dos['CEDULA_COD'].nunique(), ' personas que tienen dos sectores')
print('Se tiene un total ',afi_tres['CEDULA_COD'].nunique(), ' personas que tienen tres sectores')
print('Se tiene un total ',afi_mas['CEDULA_COD'].nunique(), ' personas que tienen más de tres sectores')

Se tiene un total  556782  personas que se va analizar su HL
Se tiene un total  391458  personas que tienen un único sector
Se tiene un total  136187  personas que tienen dos sectores
Se tiene un total  26115  personas que tienen tres sectores
Se tiene un total  3022  personas que tienen más de tres sectores


In [25]:
391458+136187+26115+3022

556782

In [26]:
#Primero se va a trabajar con las cedúlas que tiene un único sector
afi_uni['SECTOR'].unique()

array(['PRIVADO', 'PUBLICO', 'INDEPENDIENTES',
       '02-TRABAJ. AUT Y SIN RELAC DEPEN DESDE  2011-12 SOLO AF.VOLUNT  ',
       '90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA',
       '89-AFILIACION DOMICILIADO EN EL EXTERIOR',
       '108-ARTISTA Y GESTOR DE CULTURA', 'PASANTE PRIVADO',
       '69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ',
       'PASANTE PUBLICO'], dtype=object)

In [39]:
afi_uni[afi_uni['CEDULA_COD']=='16021285']['SECTOR'].unique()

array(['PUBLICO'], dtype=object)

In [40]:
#Se desagrea por sector 
pri=afi_uni[afi_uni['SECTOR']=='PRIVADO']
pub=afi_uni[afi_uni['SECTOR']=='PUBLICO']
ind=afi_uni[afi_uni['SECTOR']=='INDEPENDIENTES']
# vol_ext=afi[(afi['SECTOR']=='69-VOLUNTARIO ECUATORIANO DOMICILIADO EN EL EXTERIOR ') |
#             afi['SECTOR']=='89-AFILIACION DOMICILIADO EN EL EXTERIOR']
# vol_ecu=afi[afi['SECTOR']=='90-AFILIACION DOMICILIADO EN EL PAIS / SIN RELACION DEPENDENCIA']

In [41]:
#Sector privado
inicio = time.time()
pri=pri.pivot_table(index=('CEDULA_COD'), columns=('ANIO', 'MES'), values=('SALARIO'),aggfunc=lambda x: x)
fin = time.time()
print('Tiempo de ejecución:\n')
print('horas', (fin-inicio)//3600 )
print('minutos', ((fin-inicio)%3600)//60 )
print('segundos', (fin-inicio)%60 )

Tiempo de ejecución:

horas 0.0
minutos 4.0
segundos 12.34142780303955


In [42]:
pri

ANIO       1951                                      ... 2023                  \
MES          1   2   3   4   5   6   7   8   9   10  ...   4   5   6   7   8    
CEDULA_COD                                           ...                        
100018      NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
100034      NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
1000550     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
100057      NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
1000576     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
...         ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ..  ..  ..  ..   
99907       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
9992        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
99934       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
99947       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
99970       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   

ANIO                       2024  
MES         9   10  11  12   1   
CEDULA_COD                       
100018     NaN NaN NaN NaN  NaN  
100034     NaN NaN NaN NaN  NaN  
1000550    NaN NaN NaN NaN  NaN  
100057     NaN NaN NaN NaN  NaN  
1000576    NaN NaN NaN NaN  NaN  
...         ..  ..  ..  ..  ...  
99907      NaN NaN NaN NaN  NaN  
9992       NaN NaN NaN NaN  NaN  
99934      NaN NaN NaN NaN  NaN  
99947      NaN NaN NaN NaN  NaN  
99970      NaN NaN NaN NaN  NaN  

[177626 rows x 877 columns]

In [263]:
columnas_sin_nan = pri.columns[pri.loc[0].notna()][::-1]

In [119]:
#Para determinar las columnas con valores diferentes a NaN
columnas_sin_nan = pri.columns[pri.iloc[0].notna()][::-1]

In [137]:
meses = 12
grupos = [columnas_sin_nan[i:i+meses] for i in range(0, len(columnas_sin_nan), meses)]

In [149]:
pri.iloc[pri.index=='100018'].stack(0)

MES                  1       2       3       4       5       6       7   \
CEDULA_COD ANIO                                                           
100018     2004     NaN     NaN     NaN  388.04  512.76  458.55  488.61   
           2005     NaN     NaN  150.00  150.00  150.00  150.00  150.00   
           2006  160.00  160.00  251.58  327.00  327.00  327.00  327.00   
           2007  170.00  177.00  177.00  177.00  177.00  177.00  177.00   
           2008  223.52  207.00  261.98  213.80  238.44  249.44  270.78   
           2009  270.00  270.00  244.69  267.89  302.34  273.52  272.94   
           2010  304.50  289.50     NaN     NaN     NaN     NaN     NaN   
           2011     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
           2012  312.38  337.96  320.61  318.78  341.61  326.09  365.36   
           2013  326.14  406.49  356.43  324.39  345.41  343.71  339.40   
           2014  376.59  374.43  388.43  390.51  349.08  391.38  369.36   
           2015  386.60  421.34  377.64  360.82  382.12  388.85  359.71   
           2016  390.71  402.30  371.00  371.00  371.00  371.00  371.00   
           2017  381.00  450.06  381.00  401.24  381.00  382.19  401.24   
           2018  415.44  449.65  426.43  430.10  430.10  448.43  408.11   
           2019  440.15  415.21  512.47  399.00  399.00  403.99  399.00   

MES                  8       9       10      11      12  
CEDULA_COD ANIO                                          
100018     2004  419.20  513.76  475.22  262.35  262.35  
           2005  150.00  150.00  150.00  150.00  150.00  
           2006  327.00  327.00  327.00  167.00  167.00  
           2007  177.00  177.00  177.00  177.00  177.00  
           2008  260.95  215.41  247.68  315.55  268.30  
           2009  261.56  253.83  241.22  269.30  233.44  
           2010     NaN     NaN     NaN     NaN     NaN  
           2011     NaN  276.28  276.28  282.32  276.28  
           2012  330.65  328.83  292.29  292.29  312.38  
           2013  354.43  356.11  325.39  337.40  393.47  
           2014  400.59  361.82  344.59  395.20  382.28  
           2015  431.43  396.69  385.48  403.41  382.12  
           2016  371.00  371.00  371.00  415.06  375.64  
           2017  404.81  433.39  433.39  406.00  460.77  
           2018  420.33  477.75  465.53  419.10  541.29  
           2019  463.84  416.46  266.00     NaN     NaN

In [157]:
440.15+415.21+512.47+399.00+399.00+403.99+399.00+463.84+416.46+266.00+541.29+419.10

5075.51

In [205]:
base = 5
sumas_por_grupo = {}
for grupo in grupos:
    suma_grupo = pri[grupo].mean(axis=1).values[0]
    sumas_por_grupo[tuple(grupo)] = suma_grupo

# Ordenar los grupos por su suma en orden descendente
grupos_ordenados = sorted(sumas_por_grupo.items(), key=lambda x: x[1], reverse=True)[:base]
#grupos_ordenados

[(((2018, 10),
   (2018, 9),
   (2018, 8),
   (2018, 7),
   (2018, 6),
   (2018, 5),
   (2018, 4),
   (2018, 3),
   (2018, 2),
   (2018, 1),
   (2017, 12),
   (2017, 11)),
  436.5533333333333),
 (((2019, 10),
   (2019, 9),
   (2019, 8),
   (2019, 7),
   (2019, 6),
   (2019, 5),
   (2019, 4),
   (2019, 3),
   (2019, 2),
   (2019, 1),
   (2018, 12),
   (2018, 11)),
  422.9591666666667),
 (((2017, 10),
   (2017, 9),
   (2017, 8),
   (2017, 7),
   (2017, 6),
   (2017, 5),
   (2017, 4),
   (2017, 3),
   (2017, 2),
   (2017, 1),
   (2016, 12),
   (2016, 11)),
  403.33500000000004),
 (((2015, 10),
   (2015, 9),
   (2015, 8),
   (2015, 7),
   (2015, 6),
   (2015, 5),
   (2015, 4),
   (2015, 3),
   (2015, 2),
   (2015, 1),
   (2014, 12),
   (2014, 11)),
  389.0133333333333),
 (((2016, 10),
   (2016, 9),
   (2016, 8),
   (2016, 7),
   (2016, 6),
   (2016, 5),
   (2016, 4),
   (2016, 3),
   (2016, 2),
   (2016, 1),
   (2015, 12),
   (2015, 11)),
  378.87833333333333)]

In [218]:
componentes1 = [[par for par in tupla[0]] for tupla in grupos_ordenados]
componentes = [item for sublist in componentes1 for item in sublist]

clave_personalizada = lambda tupla: tupla[0] * 100 + tupla[1]

# Encontrar el par de año y mes mínimo y máximo usando la clave personalizada
par_minimo = min((min(sublista, key=clave_personalizada) for sublista in componentes1), key=clave_personalizada)
par_maximo = max((max(sublista, key=clave_personalizada) for sublista in componentes1), key=clave_personalizada)

In [256]:
pri['Base_calculada']=pd.NA
pri['Fec_Ini'] = pd.NA
pri['Fec_Fin'] = pd.NA

In [257]:
pri['Base_calculada'] = pri.iloc[pri.index=='100018'][componentes].mean(axis=1)
pri.loc[pri.index=='100018','Fec_Ini'] = pd.to_datetime('{}, {}'.format( par_maximo[0], par_maximo[1]), format='%Y, %m')
pri.loc[pri.index=='100018','Fec_Fin'] = pd.to_datetime('{}, {}'.format( par_minimo[0], par_minimo[1]), format='%Y, %m')

In [258]:
pri

ANIO       1951                                      ... 2023                  \
MES           1   2   3   4   5   6   7   8   9  10  ...    7   8   9  10  11   
CEDULA_COD                                           ...                        
100018      NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
100034      NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
1000550     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
100057      NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
1000576     NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
...         ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ..  ..  ..  ..   
9992        NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
99934       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
99947       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
99970       NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   
0           NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN  ...  NaN NaN NaN NaN NaN   

ANIO           2024 Base_calculada              Fec_Ini              Fec_Fin  
MES         12    1                                                           
CEDULA_COD                                                                    
100018     NaN  NaN     406.147833  2019-10-01 00:00:00  2014-11-01 00:00:00  
100034     NaN  NaN            NaN                 <NA>                 <NA>  
1000550    NaN  NaN            NaN                 <NA>                 <NA>  
100057     NaN  NaN            NaN                 <NA>                 <NA>  
1000576    NaN  NaN            NaN                 <NA>                 <NA>  
...         ..  ...            ...                  ...                  ...  
9992       NaN  NaN            NaN                  NaN                  NaN  
99934      NaN  NaN            NaN                  NaN                  NaN  
99947      NaN  NaN            NaN                  NaN                  NaN  
99970      NaN  NaN            NaN                  NaN                  NaN  
0          NaN  NaN            NaN                  NaN                  NaN  

[177627 rows x 880 columns]

In [ ]:
vej[vej['CEDULA_COD']==114]

In [ ]:
afi[afi['CEDULA_COD']==114]

In [ ]:
afi.loc[['114']]

In [ ]:
import pickle

# Nombre del archivo donde se guardará el objeto
nombre_archivo = 'proceso.pkl'

# Guardar el objeto en el archivo
with open(nombre_archivo, 'wb') as archivo:
    pickle.dump(pri, archivo)

In [ ]:
import pickle

# Nombre del archivo donde se guardó el objeto
nombre_archivo = 'proceso.pkl'

# Cargar el objeto desde el archivo
with open(nombre_archivo, 'rb') as archivo:
    mi_objeto_cargado = pickle.load(archivo)

print(mi_objeto_cargado)